# P&L Reconstruction and Data Quality Fixes

This notebook addresses historical data issues and reconstructs accurate P&L.

In [1]:
# Setup
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from src.data import BloombergCDSConnector, CDSDataManager
from src.models import CDSDatabase, Position, Strategy, Side

print('Modules imported successfully')

Modules imported successfully


## 1. Load Historical Data

In [2]:
# Initialize
connector = BloombergCDSConnector()
db = CDSDatabase('../data/cds_monitor.db')
manager = CDSDataManager(connector, db)

# Load 6 months of data
end_date = datetime.now()
start_date = end_date - timedelta(days=360)

# Load data for multiple series
series_list = [40, 41, 42, 43]
historical_data = {}

for series in series_list:
    ticker = connector.get_index_ticker('EU', 'IG', series, '5Y')
    key = f'S{series}_5Y'
    
    print(f'Loading {key}...')
    data = connector.get_historical_spreads(
        ticker, 
        start_date=start_date,
        end_date=end_date
    )
    
    if not data.empty:
        historical_data[key] = data
        print(f'  Loaded {len(data)} days')

print(f'\nTotal series loaded: {len(historical_data)}')

Loading S40_5Y...
  Loaded 249 days
Loading S41_5Y...
  Loaded 257 days
Loading S42_5Y...
  Loaded 257 days
Loading S43_5Y...
  Loaded 129 days

Total series loaded: 4


## 2. Data Quality Checks

In [3]:
# Check for data quality issues
for key, data in historical_data.items():
    print(f'\n{key} Quality Check:')
    
    # Missing values
    missing = data.isnull().sum().sum()
    print(f'  Missing values: {missing}')
    
    # Zero spreads
    if 'px_last' in data.columns:
        zeros = (data['px_last'] == 0).sum()
        print(f'  Zero spreads: {zeros}')
        
        # Statistics
        print(f'  Mean: {data["px_last"].mean():.2f} bps')
        print(f'  Std: {data["px_last"].std():.2f} bps')


S40_5Y Quality Check:
  Missing values: 0
  Zero spreads: 0
  Mean: 45.66 bps
  Std: 5.67 bps

S41_5Y Quality Check:
  Missing values: 0
  Zero spreads: 0
  Mean: 48.50 bps
  Std: 5.24 bps

S42_5Y Quality Check:
  Missing values: 0
  Zero spreads: 0
  Mean: 55.10 bps
  Std: 5.39 bps

S43_5Y Quality Check:
  Missing values: 0
  Zero spreads: 0
  Mean: 58.55 bps
  Std: 7.05 bps


## 3. Clean Data

In [4]:
def clean_spread_data(data):
    '''Clean and fix spread data'''
    cleaned = data.copy()
    
    # Handle missing values
    cleaned = cleaned.fillna(method='ffill').fillna(method='bfill')
    
    # Fix zero spreads
    if 'px_last' in cleaned.columns:
        cleaned.loc[cleaned['px_last'] == 0, 'px_last'] = np.nan
        cleaned['px_last'] = cleaned['px_last'].fillna(method='ffill')
    
    return cleaned

# Clean all series
cleaned_data = {}
for key, data in historical_data.items():
    cleaned_data[key] = clean_spread_data(data)
    print(f'Cleaned {key}')

Cleaned S40_5Y
Cleaned S41_5Y
Cleaned S42_5Y
Cleaned S43_5Y


C:\Users\alessandro.esposito\AppData\Local\Temp\ipykernel_18592\491350071.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cleaned = cleaned.fillna(method='ffill').fillna(method='bfill')
C:\Users\alessandro.esposito\AppData\Local\Temp\ipykernel_18592\491350071.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cleaned['px_last'] = cleaned['px_last'].fillna(method='ffill')


## 4. Reconstruct P&L

In [5]:
# Example P&L reconstruction for a historical position
if 'S43_5Y' in cleaned_data:
    data = cleaned_data['S43_5Y']

    # Assume we bought protection 30 days ago
    entry_idx = -30
    if len(data) > 30 and 'px_last' in data.columns:
        entry_spread = data['px_last'].iloc[entry_idx]
        current_spread = data['px_last'].iloc[-1]
        
        # Calculate P&L (assuming $10mm notional, 4500 DV01)
        notional = 10_000_000
        dv01 = 4500
        spread_change = current_spread - entry_spread
        pnl = -spread_change * dv01 * notional / 10000
        
        print('Historical Position P&L:')
        print(f'Entry spread: {entry_spread:.2f} bps')
        print(f'Current spread: {current_spread:.2f} bps')
        print(f'Spread change: {spread_change:.2f} bps')
        print(f'P&L: ${pnl:,.0f}')

Historical Position P&L:
Entry spread: 55.35 bps
Current spread: 50.44 bps
Spread change: -4.91 bps
P&L: $22,077,000
